In [2]:
#import the required libraries
import pandas as pd
import requests
import json
import warnings
from matplotlib import pyplot as plt
import calendar
import sqlite3

warnings.filterwarnings("ignore")

## Data Cleaning

In [41]:
#Read the data
orders_all = pd.read_csv("perf_test_orderdata/orders_all.csv")
orders_times = pd.read_csv("perf_test_orderdata/orders_times.csv")

In [42]:
#join the column to connect the two csv data
merge = pd.merge(orders_all,orders_times,how='left',on='admin_reference')

In [43]:
#drop the features which has most null values
data = merge.drop(['completed_at_x','customer_company','bill_state_name','ship_state_name','ship_company','subsite_store','campaign_code','bill_company'],axis=1)

In [44]:
#Date and time is splitted
new = data["completed_at_y"].str.split(" ", n = 1, expand = True) 
data['Date'] = new[0]
data['Time'] = new[1]

In [45]:
#Day, month and year is splitted
new = data["Date"].str.split("-", n = 2, expand = True) 
data['Year'] = new[0]
data['Month'] = new[1]
data['Day'] = new[2]

In [46]:
#drop null values
data = data.dropna()

In [47]:
#convert month number to month name
data['Month'] = data['Month'].astype(int).apply(lambda x: calendar.month_abbr[x])

In [48]:
#Date and time is splitted
new = data["Time"].str.split(":", n = 1, expand = True) 
data['Hour'] = new[0]
data['Minute'] = new[1]

data['Date'] = pd.to_datetime(data['Date'])  # Step 1
data['DayofWeek'] =data['Date'].dt.day_name()  # Step 2

In [49]:
#currency, sku, quantity

In [52]:
#create a connection to a database
cnx = sqlite3.connect('data.db')

In [734]:
#covert dataframe into sql table
orders_all.to_sql(name='orders_all',con=cnx)

In [735]:
#covert dataframe into sql table
orders_times.to_sql(name='orders_times',con=cnx)

In [737]:
#covert dataframe into sql table
merge.to_sql(name='merge',con=cnx)

In [739]:
#covert dataframe into sql table
data.to_sql(name='data',con=cnx)

In [12]:
#unique sku values to extract the API data 
sku_unique = merge.sku.unique()
sku_unique = list(sku_unique)

In [1]:
# links_product = [] ['product']['href']
# current_price_id = [] ['current_price']['id']
# current_price_variant_id = [] ['current_price']['variant_id']
# current_price_product_id = [] ['current_price']['product_id']
# current_price_currency_id = [] ['current_price']['currency_id']
# current_price_price_list_id = [] ['current_price']['price_list_id']
# current_price_campaign_id = [] ['current_price']['campaign_id']
# current_price_customer_id = [] ['current_price']['customer_id']
# current_price_discount_list_id = [] ['current_price']['discount_list_id']
# current_price_builder_price_list_id = [] ['current_price']['builder_price_list_id']
# current_price_include_tax = [] ['current_price']['include_tax']
# current_price_staggered = [] ['current_price']['staggered']
# current_price_amount = [] ['current_price']['amount']
# current_price_final_amount = [] ['current_price']['final_amount']
# current_price_pre_tax_amount = [] ['current_price']['pre_tax_amount']
# current_price_after_tax_amount = [] ['current_price']['after_tax_amount']
# current_price_discount = [] ['current_price']['discount']
# current_price_discount_multiplier = [] ['current_price']['discount_multiplier']
# best_price_id = [] ['best_price']['id']
# best_price_variant_id = [] ['best_price']['variant_id']
# best_price_product_id = [] ['best_price']['product_id']
# best_price_currency_id = [] ['best_price']['currency_id']
# best_price_price_list_id = [] ['best_price']['price_list_id']
# best_price_campaign_id = [] ['best_price']['campaign_id']
# best_price_customer_id = [] ['best_price']['customer_id']
# best_price_discount_list_id = [] ['best_price']['discount_list_id']
# best_price_builder_price_list_id = [] ['best_price']['builder_price_list_id']
# best_price_include_tax = [] ['best_price']['include_tax']
# best_price_staggered = [] ['best_price']['staggered']
# best_price_amount = [] ['best_price']['amount']
# best_price_final_amount = [] ['best_price']['final_amount']
# best_price_pre_tax_amount = [] ['best_price']['pre_tax_amount']
# best_price_after_tax_amount = [] ['best_price']['after_tax_amount']
# best_price_discount = [] ['best_price']['discount']
# best_price_discount_multiplier = [] ['best_price']['discount_multiplier']
# recommended_price = [] ['recommended_price']

## Extract data from api

In [1]:
#define headers
headers = {
    "Accept" : "application/json",
    "Content-Type" : "application/json",
}

#define name with empty list 
ID= []
product_id= []
ean= []
is_master= []
name= []
weight= []
reference= []
source_owner= []
source_id= []
current_price_amount= []
best_price_amount= []
stock_available_qty = []
stock_qty = []
sku_api = []
group_name = []
group_description = []
group_properties = []
links = []
recommended_price=[]

In [99]:
try:
    # the code that can cause the error
    #API Call and extract the data
    for i in sku_unique:
        r = requests.get('https://klippkungen.se/api/v1/shop/variants/search?prices=true&groups=true&sku[]={sku}'.format(sku=i), headers=headers)
        if r.status_code == 200:
            raw_data = r.json()
            for i in raw_data["variants"]:
                if i['id'] is not None:
                    ID.append(i['id'])
                else:
                    ID.append('0')
                if i['product_id'] is not None:
                    product_id.append(i['product_id'])
                else:
                    product_id.append('0')
                if i['ean'] is not None:
                    ean.append(i['ean'])
                else:
                    ean.append('0')
                if i['is_master'] is not None:
                    is_master.append(i['is_master'])
                else:
                    is_master.append('0')
                if i['weight'] is not None:
                    weight.append(i['weight'])
                else:
                    weight.append('0')
                if i['name'] is not None:
                    name.append(i['name'])
                else:
                    name.append('0')
                if i['reference'] is not None:
                    reference.append(i['reference'])
                else:
                    reference.append('0')
                if i['source_owner'] is not None:
                    source_owner.append(i['source_owner'])
                else:
                    source_owner.append('0')
                if i['source_id'] is not None:
                    source_id.append(i['source_id'])
                else:
                    source_id.append('0')
                if i['current_price']['amount'] is not None:
                    current_price_amount.append(i['current_price']['amount'])
                else:
                    current_price_amount.append('0')
                if i['best_price']['amount'] is not None:
                    best_price_amount.append(i['best_price']['amount'])
                else:
                    best_price_amount.append('0')
                if i['stock_item']['quantity'] is not None:
                    stock_qty.append(i['stock_item']['quantity'] )
                else:
                    stock_qty.append('0')
                if i['stock_item']['available_quantity'] is not None:
                    stock_available_qty.append(i['stock_item']['available_quantity'])
                else:
                    stock_available_qty.append('0')
                if i['sku']is not None:
                    sku_api.append(i['sku'])
                else:
                    sku_api.append('0')
                if i['groups'][0]['name'] is not None:
                    group_name.append(i['groups'][0]['name'])
                else:
                    group_name.append('0')
                if i['groups'][0]['description'] is not None:
                    group_description.append(i['groups'][0]['description'])
                else:
                    group_description.append('0')
                if i['groups'][0]['properties'] is not None:
                    group_properties.append(i['groups'][0]['properties'])
                else:
                    group_properties.append('0')
except IndexError: # catch the error
    pass # pass will basically ignore it
         # and execution will continue on to whatever comes
         # after the try/except block

In [100]:
#convert the exracted data to list and then to dataframe
l = [ID,product_id,sku_api,ean,is_master,weight,name,reference,source_owner,source_id,current_price,best_price,stock_qty,stock_available_qty,sku_api,group_name,group_description,group_properties]
col_names = ['ID','product_id','sku','ean','is_master','weight','name','reference','source_owner','source_id','current_price','best_price','stock_qty','stock_available_qty','sku_api','group_name','group_description','group_properties']
df = pd.DataFrame(l).transpose()
df.columns = col_names
df

,ID,product_id,sku,ean,is_master,weight,name,reference,source_owner,source_id,current_price,best_price,stock_qty,stock_available_qty,sku_api,group_name,group_description,group_properties
0,546023,421600,COC520,5060337508995,True,12.96,Monster Lewis Hamilton 50 cl x 24 st,COC520 - Monster Lewis Hamilton 50 cl x 24 st,parent,362606,None,None,54,54,COC520,Energidryck,,{}
1,546047,421624,SÄL427025,7311674270259,True,6.75,SMAKIS APELSIN KRAV BRICK 25CL - 27 st,SÄL427025 - SMAKIS APELSIN KRAV BRICK 25CL - 2...,parent,363946,None,None,18,18,SÄL427025,web-shop-products,0,{}
2,545707,421284,DAL66646,7394269666460,True,2.2,"CHOKOSKUMBANAN DALS 2,2KG KARTONG","DAL66646 - CHOKOSKUMBANAN DALS 2,2KG KARTONG",parent,362542,None,None,77,77,DAL66646,web-shop-products,0,{}
3,544760,420351,ASB9063,8435063868014,True,1.0,REVOLVER FRUKT - 1 kg,ASB9063 - REVOLVER FRUKT - 1 kg,parent,364776,None,None,171,171,ASB9063,web-shop-products,0,{}
4,544101,419692,SKÅ5060,5411011172665,True,2.0,English Winegums 2 kg,SKÅ5060 - English Winegums 2 kg,parent,360680,None,None,264,264,SKÅ5060,Topp20,0,{}
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1482,545779,421356,CHO2112,7350015222038,True,2.4,"CHOKLY HALLON LAKRITSKULOR - 2,4 kg","CHO2112 - CHOKLY HALLON LAKRITSKULOR - 2,4 kg",parent,361340,None,None,152,152,CHO2112,web-shop-products,0,{}
1483,545960,421537,WEB3224,5701397332242,True,0.2,Mr Popgun Long Bag 200g,WEB3224 - Mr Popgun Long Bag 200g,parent,496409,None,None,61,61,WEB3224,web-shop-products,0,{}
1484,544508,420099,ER6710,7314661122975,True,2.0,Pepparshots Lakrits - 2 kg,ER6710 - Pepparshots Lakrits - 2 kg,parent,363223,None,None,356,356,ER6710,web-shop-products,0,{}
1485,548552,422823,HAR6407,5701090064075,True,2.6,"PIRATOS - 2,6 kg","HAR6407 - PIRATOS - 2,6 kg",parent,360538,None,None,161,161,HAR6407,None,None,None


In [104]:
# Products data
df.to_csv('products.csv', index=False)

In [60]:
#covert dataframe into sql table
data.to_sql(name='orders',con=cnx)

In [71]:
orders = data
orders.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 862305 entries, 1 to 886932
Data columns (total 24 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   admin_reference        862305 non-null  object        
 1   state                  862305 non-null  object        
 2   payment_state          862305 non-null  object        
 3   shipment_state         862305 non-null  object        
 4   total                  862305 non-null  float64       
 5   currency               862305 non-null  object        
 6   bill_city              862305 non-null  object        
 7   bill_zipcode           862305 non-null  object        
 8   bill_country_iso_name  862305 non-null  object        
 9   ship_city              862305 non-null  object        
 10  ship_zipcode           862305 non-null  object        
 11  ship_country_iso_name  862305 non-null  object        
 12  product_name           862305 non-null  obje

In [72]:
products = df
products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 745 entries, 0 to 744
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   ID                   745 non-null    object
 1   product_id           745 non-null    object
 2   sku                  745 non-null    object
 3   ean                  745 non-null    object
 4   is_master            745 non-null    object
 5   weight               745 non-null    object
 6   name                 745 non-null    object
 7   reference            745 non-null    object
 8   source_owner         745 non-null    object
 9   source_id            745 non-null    object
 10  current_price        0 non-null      object
 11  best_price           0 non-null      object
 12  stock_qty            745 non-null    object
 13  stock_available_qty  745 non-null    object
 14  sku_api              745 non-null    object
 15  group_name           744 non-null    object
 16  group_de

In [73]:
products.head()

,ID,product_id,sku,ean,is_master,weight,name,reference,source_owner,source_id,current_price,best_price,stock_qty,stock_available_qty,sku_api,group_name,group_description,group_properties
0,546023,421600,COC520,5060337508995,True,12.96,Monster Lewis Hamilton 50 cl x 24 st,COC520 - Monster Lewis Hamilton 50 cl x 24 st,parent,362606,None,None,54,54,COC520,Energidryck,,{}
1,546047,421624,SÄL427025,7311674270259,True,6.75,SMAKIS APELSIN KRAV BRICK 25CL - 27 st,SÄL427025 - SMAKIS APELSIN KRAV BRICK 25CL - 2...,parent,363946,None,None,18,18,SÄL427025,web-shop-products,0,{}
2,545707,421284,DAL66646,7394269666460,True,2.2,"CHOKOSKUMBANAN DALS 2,2KG KARTONG","DAL66646 - CHOKOSKUMBANAN DALS 2,2KG KARTONG",parent,362542,None,None,77,77,DAL66646,web-shop-products,0,{}
3,544760,420351,ASB9063,8435063868014,True,1.0,REVOLVER FRUKT - 1 kg,ASB9063 - REVOLVER FRUKT - 1 kg,parent,364776,None,None,171,171,ASB9063,web-shop-products,0,{}
4,544101,419692,SKÅ5060,5411011172665,True,2.0,English Winegums 2 kg,SKÅ5060 - English Winegums 2 kg,parent,360680,None,None,264,264,SKÅ5060,Topp20,0,{}


In [78]:
#join the column to connect the two csv data
data = pd.merge(orders,products,how='left',on='sku')
data.head()

,admin_reference,state,payment_state,shipment_state,total,currency,bill_city,bill_zipcode,bill_country_iso_name,ship_city,...,source_owner,source_id,current_price,best_price,stock_qty,stock_available_qty,sku_api,group_name,group_description,group_properties
0,O160651894,complete,paid,ready,97.21,NOK,Hafrsfjord,4048,NO,Hafrsfjord,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,O160651894,complete,paid,ready,97.21,NOK,Hafrsfjord,4048,NO,Hafrsfjord,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,O082676927,complete,paid,shipped,435.00,SEK,Vrigstad,57697,SE,Vrigstad,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,O082676927,complete,paid,shipped,435.00,SEK,Vrigstad,57697,SE,Vrigstad,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,O082676927,complete,paid,shipped,435.00,SEK,Vrigstad,57697,SE,Vrigstad,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [105]:
# Products data
orders.to_csv('orders.csv', index=False)

In [98]:
data['group_name']

11        web-shop-products
14             BLACK FRIDAY
15             BLACK FRIDAY
16        web-shop-products
17        web-shop-products
                ...        
862284          Höstkampanj
862285    web-shop-products
862298    web-shop-products
862299    web-shop-products
862302    web-shop-products
Name: group_name, Length: 196425, dtype: object

In [84]:
data = data.drop(['current_price','best_price'],axis=1)

In [87]:
data = data.dropna()

In [89]:
data.head(2)

,admin_reference,state,payment_state,shipment_state,total,currency,bill_city,bill_zipcode,bill_country_iso_name,ship_city,...,name,reference,source_owner,source_id,stock_qty,stock_available_qty,sku_api,group_name,group_description,group_properties
11,O540502498,complete,paid,shipped,326.0,SEK,Alvesta,34234,SE,Alvesta,...,Stor hemlig Ät snart-box,JULKLAPP003 - Stor hemlig Ät snart-box,parent,497841,337,337,JULKLAPP003,web-shop-products,0,{}
14,O571518989,complete,paid,shipped,785.0,SEK,Gunnebo,59375,SE,Gunnebo,...,Grillchips 40 g - 27st,EST10132 - Grillchips 40 g - 27st,parent,364151,38,38,EST10132,BLACK FRIDAY,0,{}
